<a href="https://colab.research.google.com/github/Habtamuyihun561/MAIA/blob/main/AdaBoostClassifier.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install catboost

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 98.2/98.2 MB 5.4 MB/s eta 0:00:00


In [2]:
# Import necessary libraries
import os
import cv2
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.model_selection import train_test_split, cross_val_score, StratifiedKFold
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.decomposition import PCA
import seaborn as sns
from scipy.stats import shapiro, probplot, anderson
from sklearn.preprocessing import RobustScaler, OneHotEncoder

from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.impute import SimpleImputer
from sklearn.utils import resample
from sklearn.impute import SimpleImputer, KNNImputer
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer

##
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier

# advanced classifiers
from xgboost import XGBClassifier
import lightgbm as lgb
from catboost import CatBoostClassifier
from keras.models import Sequential
from keras.layers import Dense
from keras.utils import to_categorical
from sklearn.ensemble import AdaBoostClassifier
from sklearn.ensemble import ExtraTreesClassifier
#
from sklearn.ensemble import AdaBoostClassifier
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.ensemble import VotingClassifier
from sklearn.ensemble import StackingClassifier

In [4]:
import pandas as pd
import numpy as np
import os
from sklearn.ensemble import AdaBoostClassifier
from sklearn.model_selection import StratifiedKFold, GridSearchCV, cross_val_predict
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score, log_loss

# Define the error cost matrix
error_cost_matrix = np.array([
    [0, 1, 2],
    [1, 0, 1],
    [2, 1, 0]
])

# Function to calculate the custom error cost
#def calculate_error_cost(y_true, y_pred):
    #total_cost = 0
    #for i in range(len(y_true)):
       # total_cost += error_cost_matrix[y_true[i], y_pred[i]]
    #return total_cost / len(y_true)

# Function to calculate the custom error cost
def calculate_error_cost(y_true, y_pred, cost_matrix):
    conf_matrix = confusion_matrix(y_true, y_pred)
    err = np.sum(conf_matrix * cost_matrix) / len(y_true)
    return err

# Function to train and evaluate AdaBoost Classifier with Grid Search
def evaluate_adaboost(X_train, y_train):
    model = AdaBoostClassifier(random_state=0)
    param_grid = {
        'n_estimators': [50, 100, 200],
        'learning_rate': [0.01, 0.1, 1, 10]
    }
    skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=0)
    grid_search = GridSearchCV(model, param_grid, cv=skf, scoring='accuracy')
    grid_search.fit(X_train, y_train)

    best_model = grid_search.best_estimator_
    y_pred = cross_val_predict(best_model, X_train, y_train, cv=skf)
    y_pred_proba = cross_val_predict(best_model, X_train, y_train, cv=skf, method='predict_proba')

    metrics = {
        'accuracy': accuracy_score(y_train, y_pred),
        'precision': precision_score(y_train, y_pred, average='macro', zero_division=0),
        'recall': recall_score(y_train, y_pred, average='macro'),
        'f1': f1_score(y_train, y_pred, average='macro'),
        'roc_auc': roc_auc_score(y_train, y_pred_proba, multi_class='ovr'),
        'log_loss': log_loss(y_train, y_pred_proba),
        'error_cost': calculate_error_cost(y_train.to_numpy(), y_pred,error_cost_matrix)
    }

    return metrics, grid_search.best_params_

# Directory containing the Lasso-selected datasets
input_dir = '/content/drive/My Drive/Statistical Learning/selected_datasets_lasso_without_perform'
imputation_methods = ['mean', 'median', 'mode', 'knn', 'mice']

# Process each dataset
for method in imputation_methods:
    train_file = f'{input_dir}/{method}/train_selected_lasso_without_perform_{method}.csv'

    train_df = pd.read_csv(train_file)

    # Separate features and target variable
    X_train = train_df.drop(columns=['Class'])
    y_train = train_df['Class']

    # Evaluate AdaBoost Classifier with Grid Search
    metrics, best_params = evaluate_adaboost(X_train, y_train)

    print(f"AdaBoost metrics for {method} imputation: {metrics}")
    print(f"Best parameters for {method} imputation: {best_params}")


AdaBoost metrics for mean imputation: {'accuracy': 0.495875, 'precision': 0.38537063932311644, 'recall': 0.3748637250518036, 'f1': 0.34501240625460544, 'roc_auc': 0.5488072003027591, 'log_loss': 1.0932655664619588, 'error_cost': 0.863375}
Best parameters for mean imputation: {'learning_rate': 1, 'n_estimators': 50}
AdaBoost metrics for median imputation: {'accuracy': 0.4925, 'precision': 0.41093513565417367, 'recall': 0.37254574217525455, 'f1': 0.3429480905274021, 'roc_auc': 0.5496809596515759, 'log_loss': 1.0931294823876476, 'error_cost': 0.871375}
Best parameters for median imputation: {'learning_rate': 1, 'n_estimators': 50}
AdaBoost metrics for mode imputation: {'accuracy': 0.48825, 'precision': 0.32224856435382754, 'recall': 0.3611330557338556, 'f1': 0.3177142003484796, 'roc_auc': 0.5535203708884895, 'log_loss': 1.0867161511694183, 'error_cost': 0.8815}
Best parameters for mode imputation: {'learning_rate': 0.1, 'n_estimators': 200}
AdaBoost metrics for knn imputation: {'accuracy'